## Initialization

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np

In [ ]:
EXEC_PATH = 'C:\Program Files\IBM\ILOG\CPLEX_Studio2211\cplex\bin\x64_win64\cplex.exe'

In [ ]:
model = pyo.ConcreteModel()

## Data

In [ ]:
#data
i = 5
t = 96
delta_t = 0.25
P_ch_max = 10
P_dch_max = 10

## Sets

In [ ]:
model.I = pyo.RangeSet(i) #index of the vehicles i parked
#out of use due to undetermined number of chargers model.N = pyo.RangeSet(n) #index of the chargers installed (not used at the moment)
model.T = pyo.RangeSet(t) #index of the time periods (min)

## Parameters

In [ ]:
model.C_IG = pyo.Param(initialize=C_IG)
model.C_EG = pyo.Param(initialize=C_EG)
model.E_dep = pyo.Param(model.I, initialize=lambda model, i: E_dep[i-1])
model.E_max = pyo.Param(initialize=E_max)
model.E_min = pyo.Param(initialize=E_min)
model.P_ch_max = pyo.Param(initialize=P_ch_max)
model.P_dch_max = pyo.Param(initialize=P_dch_max)
model.P_PBN_plus = pyo.Param(model.T, initialize=lambda model, t: P_PBN_plus[t-1])
model.P_PBN_minus = pyo.Param(model.T, initialize=lambda model, t: P_PBN_minus[t-1])
model.T_arv = pyo.Param(model.I, initialize=lambda model, i: T_arv[i-1])
model.T_dep = pyo.Param(model.I, initialize=lambda model, i: T_dep[i-1])
model.delta_t = pyo.Param(initialize=delta_t)


## Variables

In [ ]:
#Binary variables

model.x = pyo.Var(model.T, model.I, domain=pyo.Binary)
model.y = pyo.Var(model.T, model.I, domain=pyo.Binary)

#Non-binary variables

model.e = pyo.Var(model.T, model.I, domain=pyo.NonNegativeReals)
model.p_ch_ev = pyo.Var(model.T, model.I, domain=pyo.NonNegativeReals)
model.p_dch_ev = pyo.Var(model.T, model.I, domain=pyo.NonNegativeReals)
model.p_ev_plus = pyo.Var(model.T, domain=pyo.NonNegativeReals)
model.p_ev_minus = pyo.Var(model.T, domain=pyo.NonNegativeReals)


## Objective Function

In [ ]:
def rule_obj(mod):
    return delta_t*sum(mod.C_IG*(mod.P_PBN_plus[t]+mod.p_ev_plus[t])+mod.C_EG*(mod.P_PBN_minus[t]+mod.p_ev_minus[t])for t in mod.T)
model.obj = pyo.Objective(rule = rule_obj, sense = pyo.minimize)

## Constraints

In [ ]:
model.constraints = pyo.ConstraintList()

#Constraint 1
for t in model.T:
    for i in model.I:
        model.constraints.add(model.x[t,i]+model.y[t,i]==1)

#Constraint 2
for t in model.T:
    model.constraints.add(sum(model.p_ch_ev[t,i]*model.x[t,i] for i in model.I)==model.p_ev_plus[t])

#Constraint 3
for t in model.T:
    model.constraints.add(sum(model.p_dch_ev[t,i]*model.y[t,i] for i in model.I)==model.p_ev_minus[t])

#Constraint 4
for i in model.I:
    for t in range(model.T_arv[i], model.T_dep[i]-1):
        model.constraints.add(model.e[t+1,i]== model.e[t,i]+delta_t*(model.p_ch_ev[t,i]*model.x[t,i]-model.p_dch_ev[t,i]*model.y[t,i]))

#Constraint 5
for i in model.I:
    for t in model.T_dep[i]:
        model.constraints.add(model.e[t,i]==model.E_dep[i])

#Constraint 6
for i in model.I:
    for t in model.T:
        model.constraints.add(model.E_min<=model.e[t,i])
        model.constraints.add(model.e[t,i]<=model.E_max)

#Constraint 7
for i in model.I:
    for t in model.T:
        model.constraints.add(0<=model.p_ch_ev)
        model.constraints.add(model.p_ch_ev<=model.P_ch_max)

#Constraint 8
for i in model.I:
    for t in model.T:
        model.constraints.add(0<=model.p_dch_ev)
        model.constraints.add(model.p_dch_ev<=model.P_dch_max)

## Solvers

In [ ]:

#CPLEX
opt_cplex = pyo.SolverFactory('cplex',executable=EXEC_PATH)
opt_cplex.options['timelimit'] = 3600
opt_cplex.options['mipgap'] = 0.01
results_cplex = opt_cplex.solve(model,tee=False,report_timing=True)
print(model.obj())

#Gurobi